In [1]:
from Pegasus.api import *

In [2]:
# not required. Just to view output from certain tools.
from pathlib import Path
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
nums1 = File("nums1.txt")
nums2 = File("nums2.txt")
nums3 = File("nums3.txt")
nums4 = File("nums4.txt")

rc = ReplicaCatalog()\
    .add_replica("local", nums1, Path(".").resolve() / "nums1")\
    .add_replica("local", nums2, Path(".").resolve() / "nums2")\
    .add_replica("local", nums3, Path(".").resolve() / "nums3")\
    .add_replica("local", nums4, Path(".").resolve() / "nums4")\
    .write()

In [9]:
tc =  TransformationCatalog()

separate = Transformation(
            "separate",
            site="local",
            pfn="/home/scitech/shared-data/separate",
            is_stageable=True,
            arch=Arch.X86_64,
            os_type=OS.LINUX
)

count = Transformation(
            "count",
            site="local",
            pfn="/home/scitech/shared-data/count",
            is_stageable=True,
            arch=Arch.X86_64,
            os_type=OS.LINUX
)

tar = Transformation(
            "tar",
            site="condorpool",
            pfn="/bin/tar",
            is_stageable=False,
            arch=Arch.X86_64,
            os_type=OS.LINUX
)

tc.add_transformations(separate, count, tar)
tc.write()

In [12]:
wf = Workflow("diamomnd")

odd_nums = File("odd_nums.txt")
even_nums = File("even_nums.txt")

job_separate = Job(separate)\
                .add_args("-a", "separate", "-T", "3", "-i", nums1,nums2, nums3, nums4, "-o", odd_nums, even_nums)\
                .add_inputs(nums1, nums2, nums3, nums4)\
                .add_outputs(odd_nums, even_nums)

odd_count = File("odd_count.txt")
job_count_odd = Job(count)\
                .add_args("-a", "count", "-T", "3", "-i", odd_nums, "-o", odd_count)\
                .add_inputs(odd_nums)\
                .add_outputs(odd_count)

even_count = File("even_count.txt")
job_count_even = Job(count)\
                .add_args("-a", "count", "-T", "3", "-i", even_nums, "-o", even_count)\
                .add_inputs(even_nums)\
                .add_outputs(even_count)

tar_result = File("result.tar.gz")
job_tar = Job(tar)\
          .add_args("-a", "count", "-T", "3", "-i", even_count, odd_count, "-o", tar_result)\
          .add_inputs(even_count, odd_count)\
          .add_outputs(tar_result)
            
wf.add_jobs(job_separate, job_count_odd, job_count_even, job_tar)

In [ ]:
try:
    wf.plan(submit=True)\
        .wait()
except PegasusClientError as e:
    print(e)